<a href="https://colab.research.google.com/github/stefan262/epidemics-modelling/blob/main/BMA_curve_fitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## import

In [78]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize
import scipy.stats
import math
import random
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant
from sklearn.model_selection import train_test_split
from itertools import combinations

from mpmath import mp
mp.dps = 50

## class Model

In [79]:
class Model:
  
  def __init__(self,x,y,allowed_polynomials):
    ''' class representing a simple polynomial regression model

    Parameters
    ----------
    x: 1d numpy.array
        feature vector
    y: 1d numpy.array
        response vector
    allowed_polynomials: 1d list
        allowed degrees for polynomials of the feature vector
    '''

    self.x = x
    self.y = y
    self.allowed_polynomials = allowed_polynomials
    self.n_param = len(allowed_polynomials)
    self.likl=0
    
  def fit(self):
    ''' fits the polynomial regression using ordinary least squares (OLS) from statsmodels.regression.linear_model 
    '''

    model_features = np.column_stack([self.x**i for i in self.allowed_polynomials])
    model_regr = OLS(self.y, model_features).fit()
    self.likl = mp.exp(-model_regr.bic/2)

    return self
  
  def is_subModel(self,other):
    ''' check if a model is a subModel of another one
    Parameters
    ----------
    other: Model
        the other model that will be checked against
    Returns
    -------
    True or False
    '''

    if len(self.allowed_polynomials) >= len(other.allowed_polynomials):
      return False
    else:
      for i in range(0,len(self.allowed_polynomials)):
        if not self.allowed_polynomials[i] in other.allowed_polynomials:
          return False
      return True

## class BMA_poly

In [80]:
class BMA_poly:
  
  def __init__(self,x,y,max_degree,c_cutoff):
    '''class repressenting Byaesian model averaging for polynomial curve fitting using the method known as "Occam's window". This implementation of Occam's window 
    rejects models that are 1/c_cutoff less likely than the most likely model, and rejects new complex models that are less likely than one of their submodels. 
    The priors are assumed to be uniform for this problem.

    Parameters
    ----------
    x: 1d numpy.array
        features vector
    y: 1d numpy.array
        response vector
    max_degree: float
        maximum degree for the polynomial regression (i.e. for max_degree = 10 we can have a polynomial of up to x^10)
    c_cutoff: float
        some models will be discarded if they predict the data far less well (i.e. if max_lik/lik_A > c_cutoff then model A will be discarded)
    
    '''

    self.x = x
    self.y = y
    self.max_degree = max_degree
    self.c_cutoff = c_cutoff
    self.good_models=[]
  
  def occam_razor(self, curr_model):
    '''excludes complex models which receive less support from the data than their simpler counterparts.
    Given the list of good_models is always ordered by the number of parameters all subModels will already be included and checked
    Parameters
    ----------
    model: Model
        model that needs to be checked

    Returns
    -------
    False if the model is too complex and True if the model is good and can be included
    '''
    for a_model in self.good_models:
      if a_model.is_subModel(curr_model) and a_model.likl > curr_model.likl:
        return False

    return True 

  def fit(self):
    '''perform the Bayesian model averaging 

    Returns
    -------
    self: BMA_poly
        Returns the new BMA_poly object as an output
    '''
     
    likl_sum = 0
    max_likl = 0
    
    # iterate through all possible models by going through the number of parameters (i.e. polynomial degrees)
    for num_parameters in range(1,self.max_degree+1): 

      # list of all models with fixed num_parameters (it will be a list of numbers that represent the allowed polynomial degrees from 0->20)
      current_list = list(combinations(list(range(self.max_degree)), num_parameters))

      for allowed_polynomials in current_list:
        curr_model = Model(x,y,allowed_polynomials).fit()
        
        # decide whether to save this model or not by first comparing its likl with the max_likl and then checking if there are any better subModels
        if max_likl/curr_model.likl < self.c_cutoff and self.occam_razor(curr_model):
          self.good_models.append(curr_model)

          # update the max_likl if needed and update the list of good models
          if curr_model.likl > max_likl:
            max_likl = curr_model.likl
            for a_model in self.good_models:
              if max_likl/a_model.likl > self.c_cutoff:
                self.good_models.remove(a_model)
 
    # Return the new BMA object as an output.
    return self

## tests

In [81]:
# generate random data
sigma = np.random.uniform(0,1) # constant noise
num_poly = 5 # number of polyomials
num_data = 30 # number of data points

weights=[np.random.uniform(-1,1) for i in range(0,num_poly)]
def f(x,weights):
    y=0
    for i in range(0,len(weights)):
        y+=weights[i]*np.power(x,i)
    return y

# generate some input data and add noise
input_data=[(lambda x:(x,f(x,weights)+sigma))(np.random.uniform(-5,5)) for i in range(0,num_data)]

x_data, y_data = [[ i for i, j in input_data ],[ j for i, j in input_data ]] #unzip the tuples in input_data
x = np.array(x_data)
y = np.array(y_data)

In [82]:
result = BMA_poly(x, y, 8, 20).fit()
for i in range(0,len(result.good_models)):
  print(result.good_models[i].allowed_polynomials)

(0, 3, 4, 5)
(1, 2, 3, 4)
(0, 1, 2, 3, 4)
